difference from mapping cache, between revised and stage2

In [1]:
# NOTE: inherit kernel from "10-pubtator-results.ipynb"

In [29]:
set_b = set(b_mappings["mapping_id"].tolist())
set_bb = set(bb_mappings["mapping_id"].tolist())
diff_b_bb = set_b.difference(set_bb)
diff_bb_b = set_bb.difference(set_b)
print(diff_b_bb)
print(diff_bb_b)

{1, 2, 26, 31, 32, 39, 41, 45, 47, 48, 54, 62, 66, 74, 81, 82, 85, 86, 89, 95, 97, 98, 102, 103, 104, 106, 109, 110, 112, 114, 118, 125, 127, 130, 134, 135, 137, 152, 155, 158, 175, 178, 179, 182, 183, 199, 201, 204, 215, 216, 219, 230, 233, 242, 245, 249, 252, 262, 270, 276, 278, 279, 283, 286, 287, 288, 289, 292, 294, 297, 300, 301, 305, 309, 311, 313, 315, 321, 323, 324, 325, 326, 332, 339, 341, 342, 344, 348, 350, 352, 353, 354, 355, 360, 364, 366, 373, 374, 376, 377, 380, 382, 388, 390, 396, 400, 401, 402, 403, 405, 406, 407, 408, 412, 417, 419, 421, 424, 425, 426, 427, 428, 431, 432, 437, 439, 440, 442, 443, 444, 445, 452, 488, 491, 492, 493, 494, 495, 496, 498, 503, 506, 511, 513, 517, 520, 521, 527, 531, 533, 536, 537, 538, 541, 543, 544, 548, 550, 553, 557, 558, 559, 571, 578, 581, 584, 585, 586, 593, 595, 597, 598, 605, 609, 610, 611, 613, 614, 615, 616, 617, 618, 623, 624, 625, 634, 637, 638, 639, 640, 644, 645, 647, 651, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664

In [33]:
def show_info(
    mapping_id: str,
    ebi_df: pd.DataFrame,
    efo_df: pd.DataFrame,
    ner_res: pd.DataFrame,
    this_mappings: pd.DataFrame = None,
    this_top100: pd.DataFrame = None,
    other_mappings: pd.DataFrame = None,
    other_top100: pd.DataFrame = None,
    bb_b: bool = True,
):
    ebi_row = ebi_df[ebi_df["mapping_id"] == mapping_id]
    if len(ebi_row) == 0:
        return
    # ---- general info
    ebi_label = list(ebi_row["query"])[0]
    truth_efo = {"id": list(ebi_row["full_id"])[0], "label": list(ebi_row["MAPPED_TERM_LABEL"])[0], 
                 "mapping_type": list(ebi_row["MAPPING_TYPE"])[0]}
    truth_efo_idx = efo_df.query(f"efo_id == '{truth_efo['id']}'").index.tolist()[0] + 1
    # print(truth_efo_idx)
    # ---- ents
    ents_ebi = ner_res.query("batch == 'ebi'").query(f"mapping_id == {mapping_id}")[["term", "ner_res"]].reset_index(drop=True)
    ents_efo = ner_res.query("batch == 'efo'").query(f"mapping_id == {truth_efo_idx}")[["term", "ner_res"]].reset_index(drop=True)
    # ---- mapping df
    this_df_init = this_top100\
        .query(f"mapping_id == {mapping_id}")\
        [["prediction", "score"]]\
        .merge(efo_df.rename(columns={"efo_id": "prediction"}), on=["prediction"])\
        .reset_index(drop=True)\
        .assign(idx=lambda df: df.index.tolist())
    this_df = pd.concat([
        this_df_init.iloc[[0]],
        this_df_init.query(f"prediction == '{truth_efo['id']}'")
    ]).drop_duplicates()[["idx", "efo_label", "score"]]
    other_df_init = other_top100\
        .query(f"mapping_id == {mapping_id}")\
        [["prediction", "score"]]\
        .merge(efo_df.rename(columns={"efo_id": "prediction"}), on=["prediction"])\
        .reset_index(drop=True)\
        .assign(idx=lambda df: df.index.tolist())
    other_df = pd.concat([
        other_df_init.iloc[[0]],
        other_df_init.query(f"prediction == '{truth_efo['id']}'")
    ]).drop_duplicates()[["idx", "efo_label", "score"]]
    if not bb_b:
        truth_efo_top_label = other_df["efo_label"][0]
        ents_efo_top = ner_res \
            .query("batch == 'efo'") \
            .query(f"term == '{truth_efo_top_label}'") \
            [["term", "ner_res"]] \
            .reset_index(drop=True)
    # ----
    print(f"  mapping_id: {mapping_id}, ebi_label: {ebi_label}")
    print(f"  goldstandard efo: {truth_efo}")
    print("\n")
    
    print("Identified ents: ebi")
    if len(ents_ebi) == 0:
        print("    ", None)
    else:
        print("    ", ents_ebi["ner_res"].tolist())
    print("\n")
    
    print("Identified ents: efo -- truth")
    if len(ents_efo) == 0:
        print("    ", None)
    else:
        print("    ", ents_efo["ner_res"].tolist())
    print("\n")
    
    if not bb_b:
        print("Identified ents: efo -- top")
        print("    ", ents_efo_top["ner_res"].tolist())
        print("\n")
    
    print("\n")
    print(f"this_df\n  {this_df} \n")
    print(f"other_df\n  {other_df} \n")
    print("\n------------\n")

In [34]:
MAPPING_ID_LIST = list(diff_bb_b)[:20]

for idx, mapping_id in enumerate(MAPPING_ID_LIST):
    print(f"# {idx}")
    show_info(
        mapping_id=mapping_id,
        ebi_df=ebi_df,
        efo_df=efo_df,
        ner_res=ner_res,
        this_top100=top100_bb,
        other_top100=top100_b,
    )

# 0
  mapping_id: 523, ebi_label: spinal injury
  goldstandard efo: {'id': 'http://www.ebi.ac.uk/efo/EFO_1001919', 'label': 'spinal cord injury', 'mapping_type': 'Exact'}


Identified ents: ebi
     [[{'ent_id': 'Disease_MESH_D013124', 'ent_term': 'spinal injury'}, {'ent_id': 'Disease_MESH_D058186', 'ent_term': 'injuries'}, {'ent_id': 'Disease_MESH_D014947', 'ent_term': 'injury'}, {'ent_id': 'Disease_MESH_D013122', 'ent_term': 'spinal'}, {'ent_id': 'Gene_6296', 'ent_term': 'spinal'}, {'ent_id': 'Disease_MESH_D019571', 'ent_term': 'injury'}, {'ent_id': 'Gene_18128', 'ent_term': 'injuries'}, {'ent_id': 'Gene_24525', 'ent_term': 'spinal injury'}]]


Identified ents: efo -- truth
     [[{'ent_id': 'Disease_MESH_D058186', 'ent_term': 'injuries'}, {'ent_id': 'Disease_MESH_D013119', 'ent_term': 'cord injury'}, {'ent_id': 'Disease_MESH_D014947', 'ent_term': 'injury'}, {'ent_id': 'Disease_MESH_D013118', 'ent_term': 'spinal cord'}, {'ent_id': 'Disease_OMIM_120970', 'ent_term': 'cord'}, {'ent_id'

In [28]:
MAPPING_ID_LIST = list(diff_b_bb)[:25]

for idx, mapping_id in enumerate(MAPPING_ID_LIST):
    print(f"# {idx}")
    show_info(
        mapping_id=mapping_id,
        ebi_df=ebi_df,
        efo_df=efo_df,
        ner_res=ner_res,
        this_top100=top100_b,
        other_top100=top100_bb,
        bb_b=False,
    )

# 0
  mapping_id: 1, ebi_label: gonarthrosis
  goldstandard efo: {'id': 'http://www.ebi.ac.uk/efo/EFO_0004616', 'label': 'osteoarthritis || knee', 'mapping_type': 'Broad'}


Identified ents: ebi
     [[]]
Identified ents: efo -- truth
     [[{'ent_id': 'Disease_MESH_D010003', 'ent_term': 'knee'}, {'ent_id': 'Disease_MESH_D020370', 'ent_term': 'osteoarthritis knee'}, {'ent_id': 'Disease_MESH_D000275', 'ent_term': 'osteoarthritis'}, {'ent_id': 'Gene_18933', 'ent_term': 'osteoarthritis knee'}, {'ent_id': 'Gene_597', 'ent_term': 'osteoarthritis'}]]
Identified ents: efo -- top
     [[{'ent_id': 'Disease_MESH_D010003', 'ent_term': 'osteoarthritis'}, {'ent_id': 'Disease_MESH_D000275', 'ent_term': 'osteoarthritis'}, {'ent_id': 'Gene_597', 'ent_term': 'osteoarthritis'}]]


this_df
     idx             efo_label     score
0    0  osteoarthritis, knee  0.620142
other_df
     idx             efo_label     score
0    0        osteoarthritis  0.555104
1    1  osteoarthritis, knee  0.433649

--------